In [1]:
import rasterio
import numpy as np
import rioxarray
from planetary_computer import sign
import pystac
import matplotlib.pyplot as plt
from collections import defaultdict
import json
import glob
from rasterio.plot import show
import os
from rasterio.enums import Resampling

In [4]:

def save_bands_as_stacked_tif(item_url, bands, output_dir, xmin, xmax, ymin, ymax, pre_post):
    item = pystac.Item.from_file(item_url)
    signed_item = sign(item)

    bands_data = []
    for band in bands:
        asset_href = signed_item.assets[band].href
        ds = rioxarray.open_rasterio(asset_href)

        # ds_cropped = ds.rio.clip_box(minx=xmin, miny=ymin, maxx=xmax, maxy=ymax)
        bands_data.append(ds)

    # Stack bands
    stacked_bands = np.dstack(bands_data)

    # Define output filename
    output_filename = os.path.join(output_dir, f"stacked_{pre_post}.tif")

    # Get metadata from ds
    meta = ds.meta

    # Update count value to the rasterio metadata
    meta.update(count=len(bands))

    # Save stacked raster
    with rasterio.open(output_filename, 'w', **meta) as dst:
        dst.write(stacked_bands)

    print(f"Stacked raster saved at: {output_filename}")
    
with open('Data/config.json', 'r') as json_file:
    config = json.load(json_file)
    
pre_event_url = config['pre_event_url']
post_event_url = config['post_event_url']
bands = config['bands']
xmin, xmax, ymin, ymax = config['coords']
side_length = config['side_length']
output_dir = config['output_dir']

save_bands_as_stacked_tif(pre_event_url, bands, output_dir, xmin, xmax, ymin, ymax, "Pre")

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 10980 and the array at index 4 has size 1830